In [1]:
import sys
import os

In [2]:
import datetime

In [3]:
import numpy
import scipy
import pandas

In [4]:
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(style="whitegrid", color_codes=True)

In [5]:
sys.path.append(os.path.join(os.path.abspath(os.path.join('../..')), 'src'))

In [6]:
import utils
import mysql_utils
import events_merge
import spectral_partition

In [7]:
from importlib import reload

In [141]:
reload(events_merge)

<module 'events_merge' from '/home/immersinn/gits/rssfeed_link_collector/src/events_merge.py'>

## Prelims

### Create Sequence of dates of interest

In [8]:
base = datetime.datetime.strptime('2017-02-01 00:00:00', '%Y-%m-%d %H:%M:%S')

In [9]:
date_list = [base + datetime.timedelta(hours=x) for x in range(0, 24*7, 6)]

In [10]:
date_list[:4]

[datetime.datetime(2017, 2, 1, 0, 0),
 datetime.datetime(2017, 2, 1, 6, 0),
 datetime.datetime(2017, 2, 1, 12, 0),
 datetime.datetime(2017, 2, 1, 18, 0)]

### Get MySQL Connection, Cursor

In [11]:
cnx = mysql_utils.getCnx()

In [12]:
cur = mysql_utils.getCur(cnx)

## Create Big Slice

In [23]:
docs = mysql_utils.query_docs_by_datetime(cursor=cur,
                                          start_dt=date_list[0],
                                          end_dt=date_list[0 + 8])

In [36]:
docs[['title']].head()

,title
0,FT podcast: World Weekly with Gideon Rachman
1,Developer DICE Moving on from ‘Star Wars Battl...
2,Judge Rules Video Game Publisher 2K Games Owns...
3,Dubai Airport is world's busiest for internati...
4,Woman claims she was booted from Spirit flight...


In [35]:
docs.ix[88]['summary']

'The mother of the 16-year-old claims her son was made to feel like a "freak".'

### 00: Query and Prep Data

In [14]:
details ={'summary' : {'features' : ['title', 'summary'],
                       'cutoff' : 0.15,
                       'to_binary' : False,
                       'make_symmetric' : True, 'sym_func' : lambda x,y : (x+y)/2}
         }

In [15]:
out_01 = events_merge.process_timeslice_v2(docs,
                                           details=details)
docids = out_01['summary']['doc_ids']
ts = out_01['summary']['tslice']

### 01: Generate Graph

In [16]:
tsg = scipy.sparse.csr_matrix((ts['vals'], ts['ij']))

In [17]:
# Density of matrix
tsg.nnz / tsg.shape[0] ** 2

0.027094718329275125

In [18]:
tsg.shape

(587, 587)

In [37]:
cci = numpy.where(tsg.sum(axis=1) > 0)[0]

In [41]:
rev_cci = {v : i for i,v in enumerate(list(cci))}

In [20]:
tsg_cci = tsg[cci,:][:, cci]

In [22]:
tsg_cci.shape

(557, 557)

## Grow

In [42]:
# assign nodes
rev_cci[88]

KeyError: 88